# 🎵 Classification des Genres Musicaux
## Notebook 3: Modélisation et Entraînement

**Objectif:** Entraîner et comparer différents modèles de machine learning.

---

## 1. Configuration et Imports

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

sys.path.insert(0, '..')

from src.config import Config
from src.models import ModelTrainer
from src.evaluation import Evaluator
from src.visualization import Visualizer

plt.style.use('seaborn-v0_8-whitegrid')
%matplotlib inline

# Fixer la graine aléatoire
np.random.seed(Config.RANDOM_STATE)

print("✅ Imports réussis!")

## 2. Chargement des Features

In [ ]:
# Charger les features extraites
features_path = Config.DATA_PROCESSED / Config.FEATURES_FILE

if features_path.exists():
    features_df = pd.read_csv(features_path)
    print(f"✅ Features chargées: {len(features_df)} fichiers, {len(features_df.columns)-2} caractéristiques")
else:
    print(f"❌ Fichier non trouvé: {features_path}")
    print("   Exécutez d'abord le notebook 02_feature_extraction.ipynb")

In [ ]:
features_df.head()

## 3. Préparation des Données

In [ ]:
# Initialiser le trainer
trainer = ModelTrainer()

# Préparer les données
X_train, X_val, X_test, y_train, y_val, y_test = trainer.prepare_data(features_df)

In [ ]:
# Afficher les shapes
print(f"\n📊 Dimensions des données:")
print(f"   X_train: {X_train.shape}")
print(f"   X_val:   {X_val.shape}")
print(f"   X_test:  {X_test.shape}")

## 4. Modèles Disponibles

In [ ]:
# Afficher les modèles disponibles
trainer.print_models_summary()

## 5. Entraînement de Tous les Modèles

In [ ]:
# Entraîner tous les modèles
results_df = trainer.train_all_models(X_train, y_train, X_val, y_val)

In [ ]:
# Afficher les résultats
print("\n📊 Résultats de l'entraînement:")
print(results_df.to_string(index=False))

In [ ]:
# Visualiser les résultats
fig, ax = plt.subplots(figsize=(12, 6))

x = np.arange(len(results_df))
width = 0.35

bars1 = ax.bar(x - width/2, results_df['train_accuracy'], width, label='Train', color='steelblue')
bars2 = ax.bar(x + width/2, results_df['val_accuracy'], width, label='Validation', color='coral')

ax.set_xlabel('Modèle')
ax.set_ylabel('Accuracy')
ax.set_title('Comparaison des Modèles', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(results_df['model_name'], rotation=45, ha='right')
ax.legend()
ax.set_ylim(0, 1)
ax.axhline(y=0.8, color='green', linestyle='--', alpha=0.5, label='80%')

plt.tight_layout()
plt.show()

## 6. Cross-Validation

In [ ]:
# Combiner X_train et X_val pour la cross-validation
X_combined = np.vstack([X_train, X_val])
y_combined = np.concatenate([y_train, y_val])

print(f"Données combinées: {X_combined.shape}")

In [ ]:
# Cross-validation sur les 3 meilleurs modèles
top_models = results_df.head(3)['model_name'].tolist()
print(f"\n🏆 Top 3 modèles: {top_models}")

cv_results = []
for model_name in top_models:
    result = trainer.cross_validate(model_name, X_combined, y_combined, cv=5)
    cv_results.append({
        'model': model_name,
        'cv_mean': result['cv_mean'],
        'cv_std': result['cv_std']
    })

cv_df = pd.DataFrame(cv_results)
print("\n📊 Résultats Cross-Validation:")
print(cv_df.to_string(index=False))

## 7. Optimisation des Hyperparamètres (Optionnel)

In [ ]:
# Optimiser le meilleur modèle (optionnel, peut prendre du temps)
# Décommentez pour exécuter

# best_model_name = trainer.best_model_name
# print(f"\n🔧 Optimisation de {best_model_name}...")
# tuning_results = trainer.hyperparameter_tuning(best_model_name, X_combined, y_combined)

## 8. Évaluation sur le Test Set

In [ ]:
# Évaluer le meilleur modèle
evaluator = Evaluator()

best_model = trainer.best_model_name
y_pred = trainer.predict(best_model, X_test)

# Afficher le rapport
evaluator.print_evaluation(best_model, y_test, y_pred)

In [ ]:
# Matrice de confusion
evaluator.plot_confusion_matrix(
    y_test, y_pred,
    normalize=True,
    title=f"Matrice de Confusion - {best_model}"
)
plt.show()

In [ ]:
# Rapport de classification visuel
evaluator.plot_classification_report(
    y_test, y_pred,
    title=f"Rapport de Classification - {best_model}"
)
plt.show()

## 9. Analyse des Erreurs

In [ ]:
# Paires de genres les plus confondues
confused_pairs = evaluator.get_most_confused_pairs(y_test, y_pred, top_n=5)

print("\n⚠️ Genres les plus souvent confondus:")
for real, pred, count in confused_pairs:
    print(f"   {real} → {pred}: {count} erreurs")

In [ ]:
# Visualiser les confusions
fig, ax = plt.subplots(figsize=(10, 5))

pairs = [f"{r}→{p}" for r, p, c in confused_pairs]
counts = [c for r, p, c in confused_pairs]

ax.barh(pairs, counts, color='coral')
ax.set_xlabel('Nombre d\'erreurs')
ax.set_title('Paires de Genres les Plus Confondues', fontsize=14, fontweight='bold')

for i, count in enumerate(counts):
    ax.text(count + 0.1, i, str(count), va='center')

plt.tight_layout()
plt.show()

## 10. Comparaison Finale de Tous les Modèles

In [ ]:
# Évaluer tous les modèles sur le test set
final_results = []

for model_name in trainer.trained_models.keys():
    y_pred_model = trainer.predict(model_name, X_test)
    metrics = evaluator.calculate_metrics(y_test, y_pred_model)
    
    final_results.append({
        'Model': model_name,
        'Accuracy': metrics['accuracy'],
        'Precision': metrics['precision'],
        'Recall': metrics['recall'],
        'F1-Score': metrics['f1_score']
    })

final_df = pd.DataFrame(final_results)
final_df = final_df.sort_values('Accuracy', ascending=False)

print("\n📊 Résultats Finaux sur le Test Set:")
print(final_df.to_string(index=False))

In [ ]:
# Graphique de comparaison finale
fig, ax = plt.subplots(figsize=(14, 6))

x = np.arange(len(final_df))
width = 0.2

metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
colors = ['steelblue', 'coral', 'seagreen', 'purple']

for i, (metric, color) in enumerate(zip(metrics, colors)):
    offset = (i - 1.5) * width
    ax.bar(x + offset, final_df[metric], width, label=metric, color=color, alpha=0.8)

ax.set_xlabel('Modèle')
ax.set_ylabel('Score')
ax.set_title('Comparaison Finale des Modèles', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(final_df['Model'], rotation=45, ha='right')
ax.legend()
ax.set_ylim(0, 1)
ax.axhline(y=0.8, color='green', linestyle='--', alpha=0.5)

plt.tight_layout()
plt.show()

## 11. Sauvegarde du Meilleur Modèle

In [ ]:
# Sauvegarder le meilleur modèle
save_path = trainer.save_model(trainer.best_model_name)
print(f"\n💾 Modèle sauvegardé: {save_path}")

In [ ]:
# Sauvegarder les résultats
final_df.to_csv(Config.REPORTS_DIR / 'model_comparison.csv', index=False)
print(f"📊 Résultats sauvegardés: {Config.REPORTS_DIR / 'model_comparison.csv'}")

## 12. Conclusions

### Résumé des Résultats:

- **Meilleur modèle:** [Nom du modèle]
- **Accuracy:** XX%
- **Genres bien classifiés:** Classical, Metal, ...
- **Genres souvent confondus:** Rock/Metal, Jazz/Blues, ...

### Interprétation:
- Les genres avec des caractéristiques acoustiques distinctes (classique, metal) sont mieux classifiés
- Les genres proches (rock/metal, jazz/blues) sont plus difficiles à distinguer

### Pistes d'amélioration:
1. Utiliser plus de données
2. Essayer des réseaux de neurones profonds (CNN sur spectrogrammes)
3. Feature engineering supplémentaire

In [ ]:
print("\n✅ Modélisation terminée!")
print(f"\n🏆 Meilleur modèle: {trainer.best_model_name}")
print("\n📌 Passez au notebook 04_evaluation.ipynb pour une analyse plus détaillée.")